In [1]:
HotelCity = [] # creo una lista dove inserire tutti gli hotel

In [2]:
%load_ext autotime
#Web extractor
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup as BSoup

#Loop
import time

import pandas as pd
import numpy as np
import json

# classi HTML degli elementi da prendere
classDate = 'review_item_date'
className = 'review_item_reviewer'
classNationality = 'reviewer_country'
classGiudiziPrec = 'review_item_user_review_count'
classScore = 'review-score-badge'
classTitle = 'review_item_header_content_container'
classReviewNeg = 'review_neg'
classReviewPos = 'review_pos'
classDatesoggiorno = 'review_staydate'
classTags = 'review_item_info_tags'
classHotel = 'standalone_header_hotel_link'
classCity = 'hotel_address'
idNextPageRev = 'review_next_page_link'
classnextPageHotel = 'rlp-main-pagination__btn-txt--next'
classReview = 'review_item'
classHotel2 = 'rlp-main-hotel__hotel-name-link'
classIndirizzo = 'hotel_address'
classEtà = 'user_age_group'
classNRecensioni = 'review_list_score_count'
classVoto = 'review-score-badge'
classClean = 'hotel_clean'
classComfort = 'hotel_comfort'
classLocation = 'hotel_location'
classService = 'hotel_services'
classStaff = 'hotel_staff'
classRapp = 'hotel_value'
classWifi = 'hotel_wifi'

In [9]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito') # usa modalità in incognito
options.add_experimental_option('prefs', {"profile.managed_default_content_settings.images": 2}) # non carica le immagini sul browser

driver = webdriver.Chrome(executable_path=r'C:/Users/loren/Documents/Scraping/chromedriver.exe', chrome_options=options)

url1 = 'https://www.booking.com/reviews/region/' # per le città bisogna inserire al posto di region it/city
città = 'england' # /bbs-inns da inserire per i B&B dopo regione
url2 = '.en-gb.html?aid=397594;label=gog235jc-index-it-XX-XX-unspec-it-com-L%3Ait-O%3AwindowsS10-B%3Afirefox-N%3Ayes-S%3Abo-U%3Ac-H%3As;sid=5d9a1ad15b07d4b6f7cf851b1c41c97f;offset='
NHotelPrec = str(0) # indica il numero dell'hotel precedente al primo della pagina (se 0 parte dall'hotel 1)
HomePage = url1 + città + url2 + NHotelPrec + '&'

driver.get(HomePage)

time: 12.6 s


# Codice per tutti gli hotel

In [4]:
numHotel = 5970
hotelperpagina = 30

for i in range(3390, numHotel, hotelperpagina):
    NHotelPrec = str(i)
    HomePage = url1 + città + url2 + NHotelPrec + '&'
    driver.get(HomePage)
    linkHotel = driver.find_elements_by_partial_link_text('See more reviews of ') # lista hotel della pagina    
    for j in range(len(linkHotel)): # ciclo for sulla lista di hotel
        linkHotel = driver.find_elements_by_partial_link_text('See more reviews of ') # lista hotel della pagina
        driver.get(linkHotel[j].get_attribute('href')) # vai alle recensioni dell'hotel
        html = BSoup(driver.page_source, 'html.parser') # scarica codice HTML
        hotel = {'Hotel' : html.find('a',{'class':classHotel}).text} # nome hotel
        indirizzo = html.find('p',{'class':classIndirizzo}).text.split(', ') # prendi stringa indirizzo e splittala in base a ,
        hotel['City'] = indirizzo[-3] # estrai la città
        hotel['_id'] = hotel['Hotel'] + '_' + hotel['City'] # crea un ID per ogni hotel
        if indirizzo[0].isdigit():
    #civico e indirizzo possono essere separati da , quindi se il primo è un numero prendi anche la seconda posizione della lista
            hotel['Address'] = indirizzo[0] + indirizzo[1]
        else:
            hotel['Address'] = indirizzo[0]
        hotel['ZIP Code'] = indirizzo[-2]
        hotel['Country'] = indirizzo[-1]
        punteggi = {} # creo un dizionario dei voti dell'hotel
        try: # uso try perchè se sono poche recensioni non ci sono i voti complessivi
            punteggi['Number of hotel reviews'] = int(html.find('p', {'class':classNRecensioni}).text.split()[2])
            # prende solo il numero di "basato su TOT recensioni" e trasformamelo in intero
            punteggi['Overall score'] = float(html.find('span',{'class':classVoto}).text[1:-1].replace(',', '.'))
            # cambia da , a . e trasformalo in float
            # prendo tutti i punteggi parziali
            punteggi['Cleanliness score'] = float(html.find('li',{'data-question':classClean}).text[13:-4].replace(',', '.'))
            punteggi['Comfort score'] = float(html.find('li',{'data-question':classComfort}).text[9:-4].replace(',', '.'))
            punteggi['Location score'] = float(html.find('li',{'data-question':classLocation}).text[10:-4].replace(',', '.'))
            punteggi['Facilities score'] = float(html.find('li',{'data-question':classService}).text[12:-4].replace(',', '.'))
            punteggi['Staff score'] = float(html.find('li',{'data-question':classStaff}).text[7:-4].replace(',', '.'))
            punteggi['Value for money score'] = float(html.find('li',{'data-question':classRapp}).text[17:-4].replace(',', '.'))
            hotel['Review score'] = punteggi # aggiungi il dizionario alla chiave Review score
        except AttributeError:
            pass
        try: # uso try perchè non tutti gli hotel hanno il WiFi gratuito
            punteggi['Free WiFi score'] = float(html.find('li',{'data-question':classWifi}).text[11:-4].replace(',', '.'))
        except AttributeError:
            pass   
        
        
        reviews = [] # creo una lista dove inserire recensioni
        hotel['Reviews'] = reviews # aggiungo la lista alla chiave Reviews
        while True: #ciclo infinito fino a quando non ci sono più recensioni (non so dalla prima pagine numero recensioni totali)
            html = BSoup(driver.page_source, 'html.parser')
            rev = html.findAll('li', {'class':classReview}) # lista delle recensioni in ogni pagina
            for el in rev: # ciclo su ogni recensione
                review = {} # creo un dizionario per elementi di ogni recensione
                review['Reviewed date'] = el.find('p',{'class':classDate}).text[11:-1]
                review['Name'] = el.find('span',{'itemprop':'name'}).text
                review['Nationality'] = el.find('span',{'itemprop':'nationality'}).text[1:]
                try: # non sempre è presente la fascia d'età
                    review['Age group'] = el.find('div',{'class':classEtà}).text[12:-1]
                except AttributeError:
                    pass
                review['Past user reviews'] = int(el.find('div',{'class':classGiudiziPrec}).text[1:-1].split()[0])
                try: # non sempre è presente lo score (trovato solo una recensione senza score)
                    review['Score'] = float(el.find('span',{'class':classScore}).text[1:-1].replace(',', '.'))
                except AttributeError:
                    pass
                review['Title'] = el.find('div',{'class':classTitle}).text[3:-3]
                # può essere presente anche solo uno dei due tipi di recensione
                try:
                    review['Negative review'] = el.find('p',{'class':classReviewNeg}).text[2:].replace('\n', '')
                except AttributeError:
                    pass
                try:
                    review['Positive review'] = el.find('p',{'class':classReviewPos}).text[2:].replace('\n', '')
                except AttributeError:
                    pass
                # per recensioni più vecchie non è presente data del soggiorno
                try:
                    review['Stay date'] = el.find('p',{'class':classDatesoggiorno}).text[11:-1]
                except AttributeError:
                    pass
                review['Tags'] = el.find('ul',{'class':classTags}).text[4:-2].replace('\n\n\n•', ',')
                reviews.append(review) # aggiungi alla lista reviews e quindi alla chiave Reviews la recensione
            
            try:
                # se esiste una pagina successiva delle recensione aprila
                driver.get(driver.find_element_by_id(idNextPageRev).get_attribute('href'))
            except NoSuchElementException: # se non è presente una pagina successiva delle recensioni
                HotelCity.append(hotel) # aggiungi l'hotel alla lista principale
                driver.get(HomePage) # ritorna alla pagina principale delle recensioni
                break
driver.quit()

time: 15h 8min 40s


# Codice per singola pagina da 30 hotel

Se codice si interrompe tramite i (indice della pagina delle recensioni) e j (indice dell'hotel nella pagina delle recensioni) è possibile concludere la pagina con questo codice

In [ ]:
NHotelPrec = str(5970)
HomePage = url1 + città + url2 + NHotelPrec + '&'
driver.get(HomePage)

linkHotel = driver.find_elements_by_partial_link_text('See more reviews of ')# lista hotel della pagina    
for j in range(0, 30): # ciclo for sulla lista di hotel
    linkHotel = driver.find_elements_by_partial_link_text('See more reviews of ')# lista hotel della pagina
    driver.get(linkHotel[j].get_attribute('href')) # vai alle recensioni dell'hotel
    html = BSoup(driver.page_source, 'html.parser')
    hotel = {'Hotel' : html.find('a',{'class':classHotel}).text} # nome hotel
    indirizzo = html.find('p',{'class':classIndirizzo}).text.split(', ')
    hotel['City'] = indirizzo[-3]
    hotel['_id'] = hotel['Hotel'] + '_' + hotel['City']
    if indirizzo[0].isdigit(): # per altri stati vedere stringa indirizzo
        hotel['Address'] = indirizzo[0] + indirizzo[1]
    else:
        hotel['Address'] = indirizzo[0]
    hotel['ZIP Code'] = indirizzo[-2]
    hotel['Country'] = indirizzo[-1]
    punteggi = {}
    try:
        punteggi['Number of hotel reviews'] = int(html.find('p', {'class':classNRecensioni}).text.split()[2])
        # prende solo il numero di "basato su TOT recensioni" e trasformamelo in intero
        punteggi['Overall score'] = float(html.find('span',{'class':classVoto}).text[1:-1].replace(',', '.'))
        # cambia da , a . e trasformalo in float
        punteggi['Cleanliness score'] = float(html.find('li',{'data-question':classClean}).text[13:-4].replace(',', '.'))
        punteggi['Comfort score'] = float(html.find('li',{'data-question':classComfort}).text[9:-4].replace(',', '.'))
        punteggi['Location score'] = float(html.find('li',{'data-question':classLocation}).text[10:-4].replace(',', '.'))
        punteggi['Facilities score'] = float(html.find('li',{'data-question':classService}).text[12:-4].replace(',', '.'))
        punteggi['Staff score'] = float(html.find('li',{'data-question':classStaff}).text[7:-4].replace(',', '.'))
        punteggi['Value for money score'] = float(html.find('li',{'data-question':classRapp}).text[17:-4].replace(',', '.'))
        hotel['Review score'] = punteggi
    except AttributeError:
        pass
    try:
        punteggi['Free WiFi score'] = float(html.find('li',{'data-question':classWifi}).text[11:-4].replace(',', '.'))
    except AttributeError:
        pass   
    reviews = []
    hotel['Reviews'] = reviews
    while True: # possibilità di un do-while con esecuzione della condizione
        html = BSoup(driver.page_source, 'html.parser')
        rev = html.findAll('li', {'class':classReview}) # lista delle recensioni in ogni pagina
        for el in rev:
            review = {}
            review['Reviewed date'] = el.find('p',{'class':classDate}).text[11:-1]
            review['Name'] = el.find('span',{'itemprop':'name'}).text
            review['Nationality'] = el.find('span',{'itemprop':'nationality'}).text[1:]
            try:
                review['Age group'] = el.find('div',{'class':classEtà}).text[12:-1]
            except AttributeError:
                pass
            review['Past user reviews'] = int(el.find('div',{'class':classGiudiziPrec}).text[1:-1].split()[0])
            try:
                review['Score'] = float(el.find('span',{'class':classScore}).text[1:-1].replace(',', '.'))
            except AttributeError:
                pass
            review['Title'] = el.find('div',{'class':classTitle}).text[3:-3]
            try:
                review['Negative review'] = el.find('p',{'class':classReviewNeg}).text[2:].replace('\n', '')
            except AttributeError:
                pass
            try:
                review['Positive review'] = el.find('p',{'class':classReviewPos}).text[2:].replace('\n', '')
            except AttributeError:
                pass
            try:
                review['Stay date'] = el.find('p',{'class':classDatesoggiorno}).text[11:-1]
            except AttributeError:
                pass
            review['Tags'] = el.find('ul',{'class':classTags}).text[4:-2].replace('\n\n\n•', ',')
            reviews.append(review)
        try:
            driver.get(driver.find_element_by_id(idNextPageRev).get_attribute('href'))
        except NoSuchElementException:
            HotelCity.append(hotel)
            driver.get(HomePage)
            break
driver.quit()

# Codice per singolo hotel

In [ ]:
html = BSoup(driver.page_source, 'html.parser')
hotel = {'Hotel' : html.find('a',{'class':classHotel}).text} # nome hotel
indirizzo = html.find('p',{'class':classIndirizzo}).text.split(', ')
hotel['City'] = indirizzo[-3]
hotel['_id'] = hotel['Hotel'] + '_' + hotel['City']
if indirizzo[0].isdigit(): # per altri stati vedere stringa indirizzo
    hotel['Address'] = indirizzo[0] + indirizzo[1]
else:
    hotel['Address'] = indirizzo[0]
hotel['ZIP Code'] = indirizzo[-2] # vale solo per Inghilterra
hotel['Country'] = indirizzo[-1]
punteggi = {}
try:
    punteggi['Number of hotel reviews'] = int(html.find('p', {'class':classNRecensioni}).text.split()[2])
    # prende solo il numero di "basato su TOT recensioni" e trasformamelo in intero
    punteggi['Overall score'] = float(html.find('span',{'class':classVoto}).text[1:-1].replace(',', '.'))
    # cambia da , a . e trasformalo in float
    punteggi['Cleanliness score'] = float(html.find('li',{'data-question':classClean}).text[13:-4].replace(',', '.'))
    punteggi['Comfort score'] = float(html.find('li',{'data-question':classComfort}).text[9:-4].replace(',', '.'))
    punteggi['Location score'] = float(html.find('li',{'data-question':classLocation}).text[10:-4].replace(',', '.'))
    punteggi['Facilities score'] = float(html.find('li',{'data-question':classService}).text[12:-4].replace(',', '.'))
    punteggi['Staff score'] = float(html.find('li',{'data-question':classStaff}).text[7:-4].replace(',', '.'))
    punteggi['Value for money score'] = float(html.find('li',{'data-question':classRapp}).text[17:-4].replace(',', '.'))
    hotel['Review score'] = punteggi
except AttributeError:
    pass
try:
    punteggi['Free WiFi score'] = float(html.find('li',{'data-question':classWifi}).text[11:-4].replace(',', '.'))
except AttributeError:
    pass   
reviews = []
hotel['Reviews'] = reviews
while True: # possibilità di un do-while con esecuzione della condizione
    html = BSoup(driver.page_source, 'html.parser')
    rev = html.findAll('li', {'class':classReview}) # lista delle recensioni in ogni pagina
    for el in rev:
        review = {}
        review['Reviewed date'] = el.find('p',{'class':classDate}).text[11:-1]
        review['Name'] = el.find('span',{'itemprop':'name'}).text
        review['Nationality'] = el.find('span',{'itemprop':'nationality'}).text[1:]
        try:
            review['Age group'] = el.find('div',{'class':classEtà}).text[12:-1]
        except AttributeError:
            pass
        review['Past user reviews'] = int(el.find('div',{'class':classGiudiziPrec}).text[1:-1].split()[0])
        try:
            review['Score'] = float(el.find('span',{'class':classScore}).text[1:-1].replace(',', '.'))
        except AttributeError:
            pass
        review['Title'] = el.find('div',{'class':classTitle}).text[3:-3]
        try:
            review['Negative review'] = el.find('p',{'class':classReviewNeg}).text[2:].replace('\n', '')
        except AttributeError:
            pass
        try:
            review['Positive review'] = el.find('p',{'class':classReviewPos}).text[2:].replace('\n', '')
        except AttributeError:
            pass
        try:
            review['Stay date'] = el.find('p',{'class':classDatesoggiorno}).text[11:-1]
        except AttributeError:
            pass
        review['Tags'] = el.find('ul',{'class':classTags}).text[4:-2].replace('\n\n\n•', ',')
        reviews.append(review)
    try:
        driver.get(driver.find_element_by_id(idNextPageRev).get_attribute('href'))
    except NoSuchElementException:
        HotelCity.append(hotel)
        break

# Controllo File

Controllo indice dell'hotel della pagina e indice della pagina

In [11]:
print(j, i)

27 5940
time: 2 ms


### Controllo numero di Hotel

In [16]:
NHotelPrec = str(0)
HomePage = url1 + città + url2 + NHotelPrec + '&'

driver = webdriver.Chrome(executable_path=r'C:/Users/loren/Documents/Scraping/chromedriver.exe', chrome_options=options)
driver.get(HomePage)

time: 5.08 s


In [17]:
pagineHotel = 3
hotel = 0
for i in range(pagineHotel):
    linkHotel = driver.find_elements_by_partial_link_text('See more reviews of ')
    hotel=hotel+len(linkHotel)
    try:
        driver.get(driver.find_element_by_class_name(classnextPageHotel).get_attribute('href'))
        # vai avanti con la pagina hotel
    except NoSuchElementException:
        break
driver.quit()
hotel

71

time: 5.25 s


### Controllo recensioni per ogni hotel

In [12]:
# Dataframe con numero recensioni per ogni hotel
d = pd.DataFrame(data={'_id':[el['_id'] for el in HotelCity],
                 'Reviews':[len(HotelCity[i]['Reviews']) for i in range(len(HotelCity))]})
d

,ID,Reviews
0,Revelstoke Hotel_Bridlington,147
1,Links Country Park Hotel_Cromer,392
2,Hillbark Hotel_Wirral,46
3,Best Western Stoke East_Stoke on Trent,525
4,The Wakefield House Hotel_Blackpool,65
5,The Imperial Crown Hotel_Halifax,605
6,Warwick Conferences - Arden_Coventry,75
7,Holiday Inn Leicester City_Leicester,1043
8,Holiday Inn Chester South_Chester,1272
9,Royal Lodge_Symonds Yat,419


time: 44 ms


### Controllo Hotel con 0 recensioni

In [13]:
# d[d['Reviews']==0].index.tolist() mi ridà la lista delle posizioni con reviews = 0
d[d['Reviews']==0]

,ID,Reviews


time: 4.99 ms


##### Cancella hotel con 0 recensioni
Ho creato una lista (listdel) che mi toglie l'indice della lista ad ogni elemento della lista siccome ogni volta che si elimina l'elemento dalla lista HotelCity l'indice scala di 1

In [21]:
#listdel = [el - d[d['Reviews']==0].index.tolist().index(el) for el in d[d['Reviews']==0].index.tolist()]

#for i in listdel:
#    del HotelCity[i]

Error in callback <function LineWatcher.stop at 0x0000021F63510488> (for post_run_cell):


AssertionError: 

### Controllo se ci sono hotel ripetuti

In [14]:
count = d.groupby('_id', as_index=False).size().to_frame('Count').sort_values(by='Count', ascending=False)
double = count[count['Count'] > 1]
double

,Count
ID,


time: 9.99 ms


Dataset degli hotel ripetuti (per individuare indice nel JSON)

In [21]:
d[d.duplicated(subset='_id', keep=False)].sort_values('_id')

,ID,Reviews


time: 66 ms


In [14]:
#del HotelCity[16]

time: 31.3 ms


# Salvataggio File

In [ ]:
with open('Dataset Hotel\HotelEngland_3410-6000.json', 'w', encoding='utf-8') as outfile:
    json.dump(HotelCity, outfile, sort_keys=True, indent=4, ensure_ascii=False)